# KITTI Dataset augmentation visualization

## Dataset structure

```
📦gta
┗ 📂training
  ┣ 📂calib
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┣ 📂image_2
  ┃ ┣ 📜<file_id>.png
  ┃ ┗ 📜 ...
  ┣ 📂label_2
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┗ 📂velodyne
    ┣ 📜<file_id>.bin
    ┗ 📜 ...
```

In [4]:
import os
import mayavi.mlab as mlab
import numpy as np
from copy import deepcopy
import pickle
import logging
import math

import pcdet.datasets.augmentor

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
from pcdet.models import build_network
from tools.visual_utils import visualize_utils as V

from PIL import Image, ImageDraw

mlab.init_notebook()

Notebook initialized with ipy backend.


In [5]:
# generate pickles from the kitti data python -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos tools/cfgs/dataset_configs/kitti_dataset.yaml
# os.system("")

## Visualizing the LiDAR point cloud with labels

In [6]:
logger = common_utils.create_logger(log_level=logging.INFO)

In [9]:
cfg_from_yaml_file('tools/cfgs/dataset_configs/kitti_dataset.yaml', cfg)
# cfg_from_yaml_file('tools/cfgs/kitti_models/pointpillar_augs.yaml', cfg)

cfg.DATA_PATH = 'data/kitti'

train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg,
    class_names=['Car', 'Pedestrian', 'Cyclist'],
    batch_size=1,
    dist=False,
    workers=4,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=False,
    total_epochs=0
)

logger.info(f'Total number of samples: \t{len(train_set)}')

data_dict_list = []
logger.info('Loading samples')
for idx, data_dict in enumerate(train_set):
    logger.info(f'Loaded sample index: \t{idx + 1}')
    data_dict = train_set.collate_batch([data_dict])
    data_dict_list.append(data_dict)
    #if idx >= 35: break


2021-12-03 12:56:10,137   INFO  Database filter by min points Car: 302 => 253
2021-12-03 12:56:10,138   INFO  Database filter by difficulty Car: 253 => 214
2021-12-03 12:56:10,139   INFO  Database filter by difficulty Pedestrian: 46 => 45
2021-12-03 12:56:10,139   INFO  Database filter by difficulty Cyclist: 17 => 12
2021-12-03 12:56:10,194   INFO  Loading KITTI dataset
2021-12-03 12:56:10,202   INFO  Total samples for KITTI dataset: 100
2021-12-03 12:56:10,203   INFO  Total number of samples: 	100
2021-12-03 12:56:10,204   INFO  Loading samples
2021-12-03 12:56:10,291   INFO  Loaded sample index: 	1
2021-12-03 12:56:10,346   INFO  Loaded sample index: 	2
2021-12-03 12:56:10,393   INFO  Loaded sample index: 	3
2021-12-03 12:56:10,431   INFO  Loaded sample index: 	4
2021-12-03 12:56:10,482   INFO  Loaded sample index: 	5
2021-12-03 12:56:10,534   INFO  Loaded sample index: 	6
2021-12-03 12:56:10,597   INFO  Loaded sample index: 	7
2021-12-03 12:56:10,655   INFO  Loaded sample index: 	8


In [98]:
scene = data_dict_list[8]

In [11]:
def show_pc(data_dict):
    currfig = mlab.figure(size=(1280,720))

    V.draw_scenes(
        points=data_dict['points'][:, 1:], gt_boxes=data_dict['gt_boxes'][0]
    )

    cam = currfig.scene.camera
    cam.zoom(2)
    return currfig


In [8]:
def show_bev(data_dict):
    currfig = mlab.figure(size=(1280,1280))

    V.draw_scenes(
        points=data_dict['points'][:, 1:], gt_boxes=data_dict['gt_boxes'][0]
    )

    mlab.view(azimuth=-179, elevation=0, distance=120.0, roll=90.0)
    
    cam = currfig.scene.camera
    cam.zoom(2)

    return currfig

In [13]:
for idx, scene in enumerate(data_dict_list):
    currfig = show_bev(scene)
    mlab.savefig('figs/mosaic/mosaic_%d.png' % idx, size=(1280,1280))
    logger.info('Saved mosaic_%d.png' % idx)


2021-11-23 16:35:33,941   INFO  Saved mosaic_0.png
2021-11-23 16:35:33,941   INFO  Saved mosaic_0.png
2021-11-23 16:35:36,253   INFO  Saved mosaic_1.png
2021-11-23 16:35:36,253   INFO  Saved mosaic_1.png
2021-11-23 16:35:38,655   INFO  Saved mosaic_2.png
2021-11-23 16:35:38,655   INFO  Saved mosaic_2.png
2021-11-23 16:35:41,100   INFO  Saved mosaic_3.png
2021-11-23 16:35:41,100   INFO  Saved mosaic_3.png
2021-11-23 16:35:43,731   INFO  Saved mosaic_4.png
2021-11-23 16:35:43,731   INFO  Saved mosaic_4.png
2021-11-23 16:35:46,132   INFO  Saved mosaic_5.png
2021-11-23 16:35:46,132   INFO  Saved mosaic_5.png
2021-11-23 16:35:49,183   INFO  Saved mosaic_6.png
2021-11-23 16:35:49,183   INFO  Saved mosaic_6.png
2021-11-23 16:35:52,240   INFO  Saved mosaic_7.png
2021-11-23 16:35:52,240   INFO  Saved mosaic_7.png
2021-11-23 16:35:55,626   INFO  Saved mosaic_8.png
2021-11-23 16:35:55,626   INFO  Saved mosaic_8.png
2021-11-23 16:35:58,527   INFO  Saved mosaic_9.png
2021-11-23 16:35:58,527   INFO 

In [ ]:
collage = Image.new("RGBA", (3000,3000), color=(255,255,255,255))

c=0
for i in range(0,3000,500):
    for j in range(0,3000,500):
        file = 'figs/mosaic/mosaic_%d.png' % c
        photo = Image.open(file).convert("RGBA")
        photo = photo.resize((500,500))        
        
        collage.paste(photo, (i,j))
        c+=1
collage.save('figs/mosaic/collage.png')

In [7]:
currfig = show_bev(data_dict_list[2])
mlab.savefig('3d_renders/original.obj')
mlab.savefig('figs/original.png', size=(1280,720))
currfig

NameError: name 'show_bev' is not defined

## Visualizing the original scene
![Original Camera Image](data/kitti/training/image_2/000007.png)


In [97]:
currfig = show_pc(data_dict_list[7])
mlab.savefig('3d_renders/aug_pipeline_output.obj')
mlab.savefig('figs/aug_pipeline_output.png', size=(1280,720))
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

## Augmentation methods

### Random flip

In [12]:
def random_flip_along_x(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 1] = -gt_boxes[:, 1]
        gt_boxes[:, 6] = -gt_boxes[:, 6]
        points[:, 1] = -points[:, 1]

        if gt_boxes.shape[1] > 8:
            gt_boxes[:, 8] = -gt_boxes[:, 8]

    return gt_boxes, points

def random_flip_along_y(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 0] = -gt_boxes[:, 0]
        gt_boxes[:, 6] = -(gt_boxes[:, 6] + np.pi)
        points[:, 0] = -points[:, 0]

        if gt_boxes.shape[1] > 7:
            gt_boxes[:, 7] = -gt_boxes[:, 7]

    return gt_boxes, points

In [13]:
%%time
flipped_y = deepcopy(scene)
flipped_y['gt_boxes'][0], flipped_y['points'][:,1:] = random_flip_along_y(flipped_y['gt_boxes'][0], flipped_y['points'][:,1:])

CPU times: user 692 µs, sys: 906 µs, total: 1.6 ms
Wall time: 914 µs


In [95]:
currfig = show_pc(flipped_y)
mlab.savefig('figs/flipped_y.png', size=(1280,720))
mlab.savefig('3d_renders/flipped_y.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [88]:
%%time
flipped_x = deepcopy(scene)
flipped_x['gt_boxes'][0], flipped_x['points'][:,1:] = random_flip_along_x(flipped_x['gt_boxes'][0], flipped_x['points'][:,1:])

CPU times: user 1.83 ms, sys: 469 µs, total: 2.3 ms
Wall time: 2.77 ms


In [93]:
currfig = show_pc(flipped_x)
mlab.savefig('figs/flipped_x.png', size=(1280,720))
mlab.savefig('3d_renders/flipped_x.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global rotation

In [17]:
def global_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
    points = common_utils.rotate_points_along_z(points[np.newaxis, :, :], np.array([noise_rotation]))[0]
    gt_boxes[:, 0:3] = common_utils.rotate_points_along_z(gt_boxes[np.newaxis, :, 0:3], np.array([noise_rotation]))[0]
    gt_boxes[:, 6] += noise_rotation
    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 7:9] = common_utils.rotate_points_along_z(
            np.hstack((gt_boxes[:, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
            np.array([noise_rotation])
        )[0][:, 0:2]

    return gt_boxes, points

In [18]:
%%time
rotated = deepcopy(scene)
rotated['gt_boxes'][0], rotated['points'][:,1:] = global_rotation(rotated['gt_boxes'][0], rotated['points'][:,1:], [-0.78539816, 0.78539816])

CPU times: user 25.1 ms, sys: 1.34 ms, total: 26.4 ms
Wall time: 14.6 ms


In [19]:
currfig = show_pc(rotated)
mlab.savefig('figs/rotated_global.png', size=(1280,720))
mlab.savefig('3d_renders/rotated_global.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global scaling

In [20]:
def global_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points
    noise_scale = np.random.uniform(scale_range[0], scale_range[1])
    points[:, :3] *= noise_scale
    gt_boxes[:, :6] *= noise_scale
    return gt_boxes, points

In [21]:
%%time
scaled = deepcopy(scene)
scaled['gt_boxes'][0], scaled['points'][:,1:] = global_scaling(scaled['gt_boxes'][0], scaled['points'][:, 1:], [0.4, 0.5])

CPU times: user 1.09 ms, sys: 1.53 ms, total: 2.62 ms
Wall time: 1.57 ms


In [22]:
currfig = show_pc(scaled)
mlab.savefig('figs/scaled_global.png', size=(1280,720))
mlab.savefig('3d_renders/scaled_global.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global translation

In [23]:
def random_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 0] += offset
    gt_boxes[:, 0] += offset

    if gt_boxes.shape[1] > 7:
        gt_boxes[:, 7] += offset

    return gt_boxes, points

def random_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 1] += offset
    gt_boxes[:, 1] += offset

    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 8] += offset

    return gt_boxes, points

def random_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 2] += offset
    gt_boxes[:, 2] += offset

    return gt_boxes, points

In [24]:
%%time
translation_x = deepcopy(scene)
translation_x['gt_boxes'][0], translation_x['points'][:,1:] = random_translation_along_x(translation_x['gt_boxes'][0], translation_x['points'][:, 1:], [15, 20])

CPU times: user 1.73 ms, sys: 0 ns, total: 1.73 ms
Wall time: 1.02 ms


In [25]:
currfig = show_pc(translation_x)
mlab.savefig('figs/translation_global_x.png', size=(1280,720))
mlab.savefig('3d_renders/translation_global_x.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [26]:
%%time
translation_y = deepcopy(scene)
translation_y['gt_boxes'][0], translation_y['points'][:,1:] = random_translation_along_y(translation_y['gt_boxes'][0], translation_y['points'][:, 1:], [15, 20])

CPU times: user 1.53 ms, sys: 1.04 ms, total: 2.57 ms
Wall time: 1.49 ms


In [27]:
currfig = show_pc(translation_y)
mlab.savefig('figs/translation_global_y.png', size=(1280,720))
mlab.savefig('3d_renders/translation_global_y.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [28]:
%%time
translation_z = deepcopy(scene)
translation_z['gt_boxes'][0], translation_z['points'][:,1:] = random_translation_along_z(translation_z['gt_boxes'][0], translation_z['points'][:, 1:], [15, 20])

CPU times: user 1.38 ms, sys: 858 µs, total: 2.24 ms
Wall time: 1.31 ms


In [29]:
currfig = show_pc(translation_z)
mlab.savefig('figs/translation_global_z.png', size=(1280,720))
mlab.savefig('3d_renders/translation_global_z.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global frustum dropout

In [30]:
def global_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 2]) - intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] < threshold]

    return gt_boxes, points

def global_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 2]) + intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] > threshold]

    return gt_boxes, points

def global_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 1]) - intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] < threshold]

    return gt_boxes, points

def global_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 1]) + intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] > threshold]

    return gt_boxes, points

In [31]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 0 ns, sys: 2.68 ms, total: 2.68 ms
Wall time: 1.9 ms


In [32]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_top.png')
mlab.savefig('3d_renders/frustum_dropout_top.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [33]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 1.5 ms, sys: 652 µs, total: 2.15 ms
Wall time: 1.33 ms


In [34]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_bottom.png')
mlab.savefig('3d_renders/frustum_dropout_bottom.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [35]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 207 µs, sys: 2.05 ms, total: 2.26 ms
Wall time: 1.51 ms


In [36]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_left.png')
mlab.savefig('figs/frustum_dropout_bottom.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [37]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 915 µs, sys: 2.46 ms, total: 3.38 ms
Wall time: 2.34 ms


In [38]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_right.png')
mlab.savefig('figs/frustum_dropout_right.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

#### Aux function to find which points are inside a bounding box

In [62]:
def get_points_in_box(points, gt_box):
    x, y, z = points[:,0], points[:,1], points[:,2]
    cx, cy, cz = gt_box[0], gt_box[1], gt_box[2]
    dx, dy, dz, rz = gt_box[3], gt_box[4], gt_box[5], gt_box[6]
    shift_x, shift_y, shift_z = x - cx, y - cy, z - cz

    MARGIN = 1e-2
    cosa, sina = math.cos(-rz), math.sin(-rz)
    local_x = shift_x * cosa + shift_y * (-sina)
    local_y = shift_x * sina + shift_y * cosa

    mask = np.logical_and(abs(shift_z) <= dz / 2.0, \
             np.logical_and(abs(local_x) <= dx / 2.0 + MARGIN, \
                 abs(local_y) <= dy / 2.0 + MARGIN ))

    points = points[mask]

    return points, mask

In [136]:
%%time
points_to_filter = deepcopy(scene)
for box in points_to_filter['gt_boxes'][0]:
    filtered_points, mask = get_points_in_box(points_to_filter['points'][:,1:], points_to_filter['gt_boxes'][0][1])

CPU times: user 2.41 ms, sys: 361 µs, total: 2.77 ms
Wall time: 4.81 ms


In [137]:
V.draw_scenes(
        points=points_to_filter['points'][:,1:][mask], gt_boxes=points_to_filter['gt_boxes'][0,1:]
    )

mlab.savefig('figs/points_in_box.png')
mlab.savefig('3d_renders/points_in_box.obj')

In [ ]:
filtered_points

array([[46.039,  7.479, -1.012,  0.9  ],
       [46.826,  8.116, -1.317,  0.   ],
       [46.142,  7.547, -0.775,  0.   ],
       [46.065,  7.187, -1.011,  0.16 ],
       [49.276,  6.82 , -1.093,  0.   ],
       [46.023,  7.625, -1.012,  0.89 ],
       [46.088,  6.894, -1.011,  0.12 ],
       [46.098,  8.159, -1.016,  0.   ],
       [46.773,  6.772, -1.028,  0.   ],
       [46.283,  7.944, -0.78 ,  0.12 ],
       [46.054,  7.333, -1.011,  0.9  ],
       [46.125,  7.099, -0.773,  0.   ],
       [46.043,  8.001, -1.014,  0.   ],
       [46.363,  6.912, -0.778,  0.25 ],
       [46.01 ,  7.846, -1.012,  0.54 ],
       [46.354,  6.985, -0.778,  0.53 ],
       [46.224,  8.083, -0.779,  0.   ],
       [48.758,  7.272, -1.373,  0.   ],
       [46.034,  7.034, -1.01 ,  0.25 ],
       [46.169,  7.254, -0.775,  0.   ]], dtype=float32)

### Local translation

In [108]:
def local_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 0] += offset

        gt_boxes[idx, 0] += offset

        if gt_boxes.shape[1] > 7:
            gt_boxes[idx, 7] += offset

    return gt_boxes, points


def local_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 1] += offset

        gt_boxes[idx, 1] += offset

        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 8] += offset

    return gt_boxes, points


def local_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 2] += offset

        gt_boxes[idx, 2] += offset

    return gt_boxes, points

In [109]:
%%time
local_translation_x = deepcopy(scene)
local_translation_x['gt_boxes'][0], local_translation_x['points'][:,1:] = local_translation_along_x(local_translation_x['gt_boxes'][0], local_translation_x['points'][:, 1:], [-5, 5])

CPU times: user 2.11 ms, sys: 2.21 ms, total: 4.32 ms
Wall time: 3.31 ms


In [110]:
currfig = show_pc(local_translation_x)
mlab.savefig('figs/translation_local_x.png', size=(1280,720))
mlab.savefig('3d_renders/translation_local_x.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [111]:
%%time
local_translation_y = deepcopy(scene)
local_translation_y['gt_boxes'][0], local_translation_y['points'][:,1:] = local_translation_along_y(local_translation_y['gt_boxes'][0], local_translation_y['points'][:, 1:], [-5, 10])

CPU times: user 0 ns, sys: 4.3 ms, total: 4.3 ms
Wall time: 3.39 ms


In [112]:
currfig = show_pc(local_translation_y)
mlab.savefig('figs/translation_local_y.png', size=(1280,720))
mlab.savefig('3d_renders/translation_local_y.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [113]:
%%time
local_translation_z = deepcopy(scene)
local_translation_z['gt_boxes'][0], local_translation_z['points'][:,1:] = local_translation_along_z(local_translation_z['gt_boxes'][0], local_translation_z['points'][:, 1:], [-5, 5])

CPU times: user 0 ns, sys: 4.9 ms, total: 4.9 ms
Wall time: 5.88 ms


In [114]:
currfig = show_pc(local_translation_z)
mlab.savefig('figs/translation_local_z.png', size=(1280,720))
mlab.savefig('3d_renders/translation_local_z.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local scaling

In [115]:
def local_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points

    for idx, box in enumerate(gt_boxes):
        noise_scale = np.random.uniform(scale_range[0], scale_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        # tranlation to axis center
        points[mask, 0] -= box[0]
        points[mask, 1] -= box[1]
        points[mask, 2] -= box[2]

        # apply scaling
        points[mask, :3] *= noise_scale

        # tranlation back to original position
        points[mask, 0] += box[0]
        points[mask, 1] += box[1]
        points[mask, 2] += box[2]

        gt_boxes[idx, 3:6] *= noise_scale
    return gt_boxes, points

In [116]:
%%time
scaled_local = deepcopy(scene)
scaled_local['gt_boxes'][0], scaled_local['points'][:,1:] = local_scaling(scaled_local['gt_boxes'][0], scaled_local['points'][:, 1:], [0.4, 0.5])

CPU times: user 7.9 ms, sys: 0 ns, total: 7.9 ms
Wall time: 7.09 ms


In [117]:
currfig = show_pc(scaled_local)
mlab.savefig('figs/scaled_local.png', size=(1280,720))
mlab.savefig('3d_renders/scaled_local.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local rotation

In [118]:
def local_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    
    for idx, box in enumerate(gt_boxes):
        noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        centroid_x = box[0]
        centroid_y = box[1]
        centroid_z = box[2]

        # tranlation to axis center
        points[mask, 0] -= centroid_x
        points[mask, 1] -= centroid_y
        points[mask, 2] -= centroid_z
        box[0] -= centroid_x
        box[1] -= centroid_y
        box[2] -= centroid_z

        # apply rotation
        points[mask, :] = common_utils.rotate_points_along_z(points[np.newaxis, mask, :], np.array([noise_rotation]))[0]
        box[0:3] = common_utils.rotate_points_along_z(box[np.newaxis, np.newaxis, 0:3], np.array([noise_rotation]))[0][0]

        # tranlation back to original position
        points[mask, 0] += centroid_x
        points[mask, 1] += centroid_y
        points[mask, 2] += centroid_z
        box[0] += centroid_x
        box[1] += centroid_y
        box[2] += centroid_z

        gt_boxes[idx, 6] += noise_rotation
        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 7:9] = common_utils.rotate_points_along_z(
                np.hstack((gt_boxes[idx, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
                np.array([noise_rotation])
            )[0][:, 0:2]

    return gt_boxes, points

In [119]:
%%time
rotated_local = deepcopy(scene)
rotated_local['gt_boxes'][0], rotated_local['points'][:,1:] = local_rotation(rotated_local['gt_boxes'][0],rotated_local['points'][:, 1:], [-0.78539815, 0.78539816])

CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 16.3 ms


In [120]:
currfig = show_pc(rotated_local)
mlab.savefig('figs/rotated_local.png', size=(1280,720))
mlab.savefig('3d_renders/rotated_local.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local frustum dropout

In [181]:
def local_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        threshold = (z + dz/2) - intensity * dz

        points = points[np.logical_not(np.logical_and(mask, points[:,2] >= threshold))]

    return gt_boxes, points

def local_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        threshold = (z - dz/2) + intensity * dz

        points = points[np.logical_not(np.logical_and(mask, points[:,2] <= threshold))]

    return gt_boxes, points

def local_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        threshold = (y + dy/2) - intensity * dy

        points = points[np.logical_not(np.logical_and(mask, points[:,1] >= threshold))]

    return gt_boxes, points

def local_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        threshold = (y - dy/2) + intensity * dy

        points = points[np.logical_not(np.logical_and(mask, points[:,1] <= threshold))]

    return gt_boxes, points

In [171]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.5, 0.5])

CPU times: user 4.3 ms, sys: 2.94 ms, total: 7.24 ms
Wall time: 6.18 ms


In [177]:
currfig = mlab.figure(size=(1280,720))

V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_top.png')
mlab.savefig('3d_renders/local_frustum_dropout_top.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [174]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 5.07 ms, sys: 143 µs, total: 5.21 ms
Wall time: 4.29 ms


In [175]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_bottom.png')
mlab.savefig('3d_renders/local_frustum_dropout_bottom.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [179]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 8.67 ms, sys: 77 µs, total: 8.75 ms
Wall time: 7.42 ms


In [180]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_left.png')
mlab.savefig('3d_renders/local_frustum_dropout_left.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [182]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 6 ms, sys: 255 µs, total: 6.25 ms
Wall time: 5.15 ms


In [183]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_right.png')
mlab.savefig('3d_renders/local_frustum_dropout_right.obj')
currfig

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…